In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import re

DB_NAME = "Cars2.db"
TABLE_NAME = "all_car_data"

def connect_db():
    try:
        conn = sqlite3.connect(DB_NAME)
        return conn
    except sqlite3.Error as e:
        print("ბაზასთან დაკავშირების შეცდომა:", e)
        return None

def show_tables():
    with connect_db() as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        print("ცხრილები ბაზაში:")
        for table in tables:
            print("-", table[0])

def fetch_high_rating():
    try:
        with connect_db() as conn:
            cursor = conn.cursor()
            cursor.execute(f"""
                SELECT * FROM {TABLE_NAME}
                WHERE CAST(SUBSTR(Rating, 1, 3) AS FLOAT) > 4.5;
            """)
            results = cursor.fetchall()
            print("მანქანები შეფასებით > 4.5:")
            for row in results:
                print(row)
    except Exception as e:
        print("შეცდომა მონაცემების წამოღებისას:", e)

def insert_car():
    try:
        brand = input("ბრენდი: ").strip()
        name = input("მოდელის სახელი: ").strip()
        price = input("ფასი (მაგ: Rs. 10.50 Lakh): ").strip()
        rating = input("შეფასება (მაგ: 4.5/5): ").strip()
        safety = input("უსაფრთხოება (მაგ: 5 Star Safety): ").strip()
        mileage = input("მოხმარება (მაგ: 18 kmpl): ").strip()
        power = input("ძალა (მაგ: 150 bhp): ").strip()

        with connect_db() as conn:
            cursor = conn.cursor()
            cursor.execute(f"""
                INSERT INTO {TABLE_NAME}
                (Brand, "Car Name", Price, Rating, Safety, Mileage, "Power (BHP)")
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (brand, name, price, rating, safety, mileage, power))
            conn.commit()
            print("✅ ჩანაწერი წარმატებით დაემატა!")
    except Exception as e:
        print("შეცდომა ჩანაწერის დამატებისას:", e)

def update_price():
    try:
        car_name = input("მოდელის სახელი: ").strip()
        new_price = input("ახალი ფასი (მაგ: Rs. 12.00 Lakh): ").strip()

        with connect_db() as conn:
            cursor = conn.cursor()
            cursor.execute(f"""
                UPDATE {TABLE_NAME}
                SET Price = ?
                WHERE "Car Name" = ?;
            """, (new_price, car_name))
            if cursor.rowcount == 0:
                print("ასეთი მოდელი ვერ მოიძებნა.")
            else:
                conn.commit()
                print(" ფასი წარმატებით განახლდა!")
    except Exception as e:
        print("შეცდომა განახლებისას:", e)

def delete_car():
    try:
        car_name = input("მოდელის სახელი წასაშლელად: ").strip()
        with connect_db() as conn:
            cursor = conn.cursor()
            cursor.execute(f"""
                DELETE FROM {TABLE_NAME}
                WHERE "Car Name" = ?;
            """, (car_name,))
            if cursor.rowcount == 0:
                print(" ასეთი მოდელი ვერ მოიძებნა.")
            else:
                conn.commit()
                print(" ჩანაწერი წაიშალა!")
    except Exception as e:
        print("შეცდომა წაშლისას:", e)

def plot_data():
    try:
        with connect_db() as conn:
            df = pd.read_sql_query(f"SELECT * FROM {TABLE_NAME}", conn)

        # ფასის გაწმენდა და გაუმართავი ჩანაწერების მოცილება
        df['Price_num'] = pd.to_numeric(
            df['Price'].str.extract(r'([\d.]+)')[0],
            errors='coerce'
        )
        df = df.dropna(subset=['Price_num'])

        # 1) ჰისტოგრამა
        plt.figure(figsize=(8, 5))
        plt.hist(df['Price_num'], bins=10, color='skyblue', edgecolor='black')
        plt.title('ფასების განაწილება')
        plt.xlabel('ფასი (ლაქში)')
        plt.ylabel('მანქანების რაოდენობა')
        plt.show()

        # 2) სვეტოვანი: ბრენდების რაოდენობა
        df['Brand'].value_counts().plot(kind='bar', color='orange', figsize=(10, 5))
        plt.title('მანქანების რაოდენობა ბრენდების მიხედვით')
        plt.xlabel('ბრენდი')
        plt.ylabel('რაოდენობა')
        plt.xticks(rotation=45)
        plt.show()

        # 3) წრიული: ყველაზე გავრცელებული მოდელები
        top_models = df['Car Name'].value_counts().head(5)
        plt.figure(figsize=(7, 7))
        plt.pie(top_models, labels=top_models.index, autopct='%1.1f%%')
        plt.title('ყველაზე გავრცელებული მოდელები')
        plt.show()

    except Exception as e:
        print("შეცდომა ვიზუალიზაციისას:", e)

# ===========================
# ძირითადი მენიუ
def main():
    while True:
        print("\n🔧 მენიუ:")
        print("1. ცხრილების ნახვა")
        print("2. მანქანები შეფასებით > 4.5")
        print("3. ახალი მანქანის დამატება")
        print("4. ფასის განახლება")
        print("5. მანქანის წაშლა")
        print("6. დიაგრამების ნახვა")
        print("0. გამოსვლა")

        choice = input("აირჩიე მოქმედება: ")
        if choice == "1":
            show_tables()
        elif choice == "2":
            fetch_high_rating()
        elif choice == "3":
            insert_car()
        elif choice == "4":
            update_price()
        elif choice == "5":
            delete_car()
        elif choice == "6":
            plot_data()
        elif choice == "0":
            print("გმადლობთ!")
            break
        else:
            print("არასწორი არჩევანი, სცადე ხელახლა.")

if __name__ == "__main__":
    main()
